In [53]:
import random
from copy import deepcopy

In [54]:
def is_feasible(solution):
    return any(solution)

In [55]:
def initialize(num_resources):
# True, False, False, False, True
    solution = [random.random() < 0.25 for _ in range(num_resources)]
    if not is_feasible(solution):
        random_resource = random.randrange(0, num_resources)
        solution[random_resource] = True
    return solution

In [56]:
def calc_solution_value(solution, cost, fixed_cost):
#     cost.shape = (num_users, num_resources)
# fixed_cost.shape = (num_resources, )
    num_users = len(cost)
    num_resources = len(fixed_cost)
    value = 0
    used_resources = [False for _ in range(num_resources)]
    for i in range(num_users):
        min_cost = float('inf')
        used_resource = -1
        for j in range(num_resources):
            if solution[j] and cost[i][j] < min_cost:
                min_cost = cost[i][j]
                used_resource = j
        value += min_cost
        used_resources[used_resource] = True
        
    for j, resource in enumerate(used_resources):
        if resource:
            value += fixed_cost[j]
            solution[j] = True
        else:
            solution[j] = False
    
    return value

In [70]:
def make_small_change(solution):
    new_solution = deepcopy(solution)
    random_resource = random.randrange(0, len(new_solution))
    new_solution[random_resource] = not new_solution[random_resource]
    if is_feasible(new_solution):
        return new_solution
    else:
#         random_resource - jedini True
        allowed_resources = set(range(0, len(new_solution))).difference({random_resource})
        new_random_resource = random.choice(list(allowed_resources))
        new_solution[new_random_resource] = not new_solution[new_random_resource]
        new_solution[random_resource] = not new_solution[random_resource]
        return new_solution

In [71]:
def local_search(cost, fixed_cost, max_iters):
    solution = initialize(len(fixed_cost))
    value = calc_solution_value(solution, cost, fixed_cost)
    
    for i in range(max_iters):
        new_solution = make_small_change(solution)
        new_value = calc_solution_value(new_solution, cost, fixed_cost)
        
        if new_value < value:
            value = new_value
            solution = deepcopy(new_solution)
    
    return solution, value

In [72]:
def read_instance(file_path):
    with open(file_path, 'r') as f:
        num_users, num_resources = [int(x) for x in f.readline().split()]
        cost = [[int(x) for x in f.readline().split()] for _ in range(num_users)]
        fixed_cost = [int(x) for x in f.readline().split()]
        return cost, fixed_cost

In [79]:
cost, fixed_cost = read_instance('uflp1.txt')
local_search(cost, fixed_cost, max_iters=100)

([True, False, False], 34)

In [90]:
def simulated_annealing(cost, fixed_cost, max_iters):
    solution = initialize(len(fixed_cost))
    value = calc_solution_value(solution, cost, fixed_cost)
    best_solution = deepcopy(solution)
    best_value = value
    
    for i in range(1, max_iters):
        new_solution = make_small_change(solution)
        new_value = calc_solution_value(new_solution, cost, fixed_cost)
        
        if new_value < value:
            value = new_value
            solution = deepcopy(new_solution)
            if new_value < best_value:
                best_value = new_value
                best_solution = deepcopy(new_solution)
        else:
            p = 1 / i ** 0.5
            q = random.random()
            if q < p:
                value = new_value
                solution = deepcopy(new_solution)
                
    return best_solution, best_value

In [98]:
simulated_annealing(cost, fixed_cost, max_iters=100)

([True, False, False], 34)

In [99]:
def shaking(solution, k):
    chosen_resources = random.sample(range(len(solution)), k)
    new_solution = deepcopy(solution)
    for resource in chosen_resources:
        new_solution[resource] = not new_solution[resource]
    if not is_feasible(new_solution):
        random_resource = random.randrange(len(solution))
        new_solution[random_resource] = True
    return new_solution

In [130]:
def vns(cost, fixed_cost, max_iters, k_max, move_prob):
    solution = initialize(len(fixed_cost))
    value = calc_solution_value(solution, cost, fixed_cost)
    
    for i in range(max_iters):
        for k in range(1, k_max):
            new_solution = shaking(solution, k)
#             new_solution = LS(new_solution)
            new_value = calc_solution_value(new_solution, cost, fixed_cost)
            
            if new_value < value or (new_value == value and random.random() < move_prob):
                value = new_value
                solution = deepcopy(new_solution)
                break
    
    return solution, value

In [131]:
vns(cost, fixed_cost, max_iters=10, k_max=2, move_prob=0.5)

([True, False, False], 34)